In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/big-mart-data/bigmart_data.csv')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.Outlet_Identifier.unique()

In [ ]:
df.head()

In [ ]:
seriesOfNullId = df.loc[df['Item_Weight'].isnull(), 'Item_Identifier']
seriesOfNullId
for x in seriesOfNullId:
    currList = df.loc[df['Item_Identifier']==x, 'Item_Weight'].unique()
    for y in currList:
        if(not np.isnan(y)):
            df.loc[df['Item_Identifier']==x,'Item_Weight'] = y


In [ ]:
df.isna().sum()

In [ ]:
df.loc[df['Item_Weight'].isnull()]

In [ ]:
df.Item_Weight.fillna(np.mean(df['Item_Weight']), inplace=True)

In [ ]:
df.corr()

In [ ]:
mode_w_size = df.pivot_table(values='Outlet_Size',columns='Outlet_Type', aggfunc= (lambda x: x.mode()[0]))


In [ ]:
df.loc[df.Outlet_Size.isnull(),'Outlet_Size'] = df.loc[df.Outlet_Size.isnull(),'Outlet_Type'].apply(lambda x: mode_w_size[x])

In [ ]:
df.loc[df.Item_Visibility==0,'Item_Visibility'] = np.mean(df.Item_Visibility)


In [ ]:
df.Outlet_Size.value_counts()

In [ ]:
df.head()

In [ ]:
# abc, cde = pd.factorize(df['Outlet_Size'])
# df['Outlet_Size']=abc

size_dict = {'Small':0, 'Medium':1, 'High':2}
df['Outlet_Size'] = df['Outlet_Size'].map(size_dict)

In [ ]:
df.head()

In [ ]:
df.Outlet_Establishment_Year.max()

In [ ]:
df['Outlet_Establishment_Year'] = 2021 - df['Outlet_Establishment_Year']

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df['Outlet_Type'].unique()

In [ ]:
tier_dict = {'Tier 1':0, 'Tier 2':1, 'Tier 3':2}
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].map(tier_dict)

type_dict = {'Grocery Store':0, 'Supermarket Type1':1, 'Supermarket Type2':2, 'Supermarket Type3':3}
df['Outlet_Type'] = df['Outlet_Type'].map(type_dict)



In [ ]:
df.corr()

In [ ]:
df.Item_Fat_Content.unique()

In [ ]:
df.Item_Fat_Content.replace(['LF','low fat'],'Low Fat', inplace=True)
df.Item_Fat_Content.replace('reg','Regular', inplace=True)

In [ ]:
df.Item_Fat_Content.unique()

In [ ]:
def returnItemType(x):
    if x=='FD':
        return 'Food'
    elif x == 'DR':
        return 'Drink'
    return 'Non Consumable'

df['new feature'] = df.Item_Identifier.apply(lambda x:  returnItemType(x[:2]))

In [ ]:
df['Item_Cat'] = df['new feature']
df.drop('new feature', axis = 1, inplace=True)

df.loc[df['Item_Cat']=='Non Consumable','Item_Fat_Content'] = 'Non Consumable'


In [ ]:
df.Item_Fat_Content.value_counts()

In [ ]:
fat_dict = {'Non Consumable':0, 'Low Fat':1, 'Regular':2}
df['Item_Fat_Content'] = df['Item_Fat_Content'].map(fat_dict)

In [ ]:
final_df = df.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)

In [ ]:
final_df

In [ ]:
sns.distplot(final_df['Item_MRP'])

In [ ]:
#0-80
#81-140
#141-200
#201>

# def returnPriceRange(price):
#     if(price<=80):
#         return 0
#     elif(price<=140):
#         return 1
#     elif(price<=200):
#         return 2
#     return 3


# final_df['mrp_range'] = final_df['Item_MRP'].apply(lambda x: returnPriceRange(x))

In [ ]:
final_df.head()

In [ ]:
final_df['Item_Outlet_Sales'] = np.log(1+df['Item_Outlet_Sales'])

In [ ]:
fdf_num = final_df[['Item_Weight','Item_Visibility','Item_MRP']]

In [ ]:
fdf_num

In [ ]:
fdf_num.corr()

In [ ]:
# final_df.drop(['Item_Weight','Item_Visibility','Item_MRP'], axis=1, inplace=True)

In [ ]:
# final_df = final_df.join(fdf_num)

In [ ]:
final_df

In [ ]:
final_df = pd.get_dummies(data=final_df,drop_first=True)

In [ ]:
final_df

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor




In [ ]:
X = final_df.drop('Item_Outlet_Sales', axis=1)
Y = final_df['Item_Outlet_Sales']

In [ ]:
# lin = LinearRegression(normalize=True)
# lin.fit(X,Y)
# pred = lin.predict(X)
# cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
# mse = mean_squared_error(Y,pred)
# print('mse: ',mse)
# print('cvs: ',np.abs(np.mean(cv_score)))


In [ ]:
lin = LinearRegression(normalize=True)
lin.fit(X,Y)
pred = lin.predict(X)
cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
mse = mean_squared_error(Y,pred)
print('mse: ',mse)
print('cvs: ',np.abs(np.mean(cv_score)))


In [ ]:
for x in [45]:
    for y in [4]:
        lin = XGBRegressor(n_estimators=x, max_depth=y,n_jobs=4)
        lin.fit(X,Y)
        pred = lin.predict(X)
        cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
        mse = mean_squared_error(Y,pred)
        print('mse for ests: ',x, ' and depth: ',y,': ',mse)
        print('cvs: ',np.abs(np.mean(cv_score)))
        print('=-=-=-=-=-=-=-=')


In [ ]:
lin = DecisionTreeRegressor(max_depth=5)
lin.fit(X,Y)
pred = lin.predict(X)
cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
mse = mean_squared_error(Y,pred)
print('mse: ',mse)
print('cvs: ',np.abs(np.mean(cv_score)))


In [ ]:
for x in [100]:
    for y in [5]:
        lin = RandomForestRegressor(max_depth=5)
        lin.fit(X,Y)
        pred = lin.predict(X)
        cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
        mse = mean_squared_error(Y,pred)
        print('mse for ests: ',x, ' and depth: ',y,': ',mse)
        print('cvs: ',np.abs(np.mean(cv_score)))
        print('=-=-=-=-=-=-=-=')

#105,5 is best

In [ ]:
lin = ExtraTreesRegressor(n_estimators=105,max_depth=5)
lin.fit(X,Y)
pred = lin.predict(X)
cv_score = cross_val_score(lin,X,Y,scoring='neg_mean_squared_error')
mse = mean_squared_error(Y,pred)
print('mse: ',mse)
print('cvs: ',np.abs(np.mean(cv_score)))


In [ ]:
# actual_predictions = np.exp(pred)
lin = XGBRegressor(n_estimators=45,max_depth=5)
lin.fit(X,Y)


In [ ]:
df = pd.read_csv('../input/big-mart-test/test_AbJTz2l.csv')

seriesOfNullId = df.loc[df['Item_Weight'].isnull(), 'Item_Identifier']
seriesOfNullId
for x in seriesOfNullId:
    currList = df.loc[df['Item_Identifier']==x, 'Item_Weight'].unique()
    for y in currList:
        if(not np.isnan(y)):
            df.loc[df['Item_Identifier']==x,'Item_Weight'] = y



df.Item_Weight.fillna(np.mean(df['Item_Weight']), inplace=True)

mode_w_size = df.pivot_table(values='Outlet_Size',columns='Outlet_Type', aggfunc= (lambda x: x.mode()[0]))


df.loc[df.Outlet_Size.isnull(),'Outlet_Size'] = df.loc[df.Outlet_Size.isnull(),'Outlet_Type'].apply(lambda x: mode_w_size[x])

df.loc[df.Item_Visibility==0,'Item_Visibility'] = np.mean(df.Item_Visibility)


size_dict = {'Small':0, 'Medium':1, 'High':2}
df['Outlet_Size'] = df['Outlet_Size'].map(size_dict)
tier_dict = {'Tier 1':0, 'Tier 2':1, 'Tier 3':2}
df['Outlet_Location_Type'] = df['Outlet_Location_Type'].map(tier_dict)
type_dict = {'Grocery Store':0, 'Supermarket Type1':1, 'Supermarket Type2':2, 'Supermarket Type3':3}
df['Outlet_Type'] = df['Outlet_Type'].map(type_dict)

df['Outlet_Establishment_Year'] = 2021 - df['Outlet_Establishment_Year']

df.Item_Fat_Content.replace(['LF','low fat'],'Low Fat', inplace=True)
df.Item_Fat_Content.replace('reg','Regular', inplace=True)

df['new feature'] = df.Item_Identifier.apply(lambda x:  returnItemType(x[:2]))


df['Item_Cat'] = df['new feature']
df.drop('new feature', axis = 1, inplace=True)

df.loc[df['Item_Cat']=='Non Consumable','Item_Fat_Content'] = 'Non Consumable'
fat_dict = {'Non Consumable':0, 'Low Fat':1, 'Regular':2}
df['Item_Fat_Content'] = df['Item_Fat_Content'].map(fat_dict)

final_df = df.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)
# final_df['mrp_range'] = final_df['Item_MRP'].apply(lambda x: returnPriceRange(x))
final_df = pd.get_dummies(data=final_df,drop_first=True)




In [ ]:
final_df

In [ ]:
pred = lin.predict(final_df)

In [ ]:
actual_predictions = np.exp(pred)

In [ ]:
output = pd.DataFrame({'Item_Identifier': df.Item_Identifier,
                       'Outlet_Identifier': df.Outlet_Identifier,
                      'Item_Outlet_Sales':actual_predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
# final_df